# Bibliotecas a serem utilizadas:

In [ ]:
import matplotlib.pyplot as plt  #Visualização de dados
import numpy as np
import seaborn as sns
import pandas as pd  #Manipular database
from sklearn.model_selection import train_test_split  #Divisor database
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix  #Acurácia dos resultados

# Dataset:

In [ ]:
datasetExt = pd.read_excel("/content/mushroomExtended_data.xlsx")
datasetExt #Base de dados com os nomes por extenso

,classe,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,EDIBLE,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,WHITE,...,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,PURPLE,SEVERAL,WOODS
1,EDIBLE,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,WHITE,...,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,BROWN,SEVERAL,WOODS
2,EDIBLE,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,PINK,...,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,PURPLE,SEVERAL,WOODS
3,EDIBLE,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,PINK,...,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,BROWN,SEVERAL,WOODS
4,EDIBLE,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,BROWN,...,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,PURPLE,SEVERAL,WOODS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8411,EDIBLE,KNOBBED,SMOOTH,BROWN,NO,NONE,ATTACHED,CLOSE,BROAD,BROWN,...,SMOOTH,ORANGE,ORANGE,PARTIAL,BROWN,ONE,PENDANT,BROWN,CLUSTERED,LEAVES
8412,EDIBLE,KNOBBED,SMOOTH,BROWN,NO,NONE,ATTACHED,CLOSE,BROAD,BROWN,...,SMOOTH,ORANGE,ORANGE,PARTIAL,BROWN,ONE,PENDANT,ORANGE,SEVERAL,LEAVES
8413,EDIBLE,KNOBBED,SMOOTH,BROWN,NO,NONE,ATTACHED,CLOSE,BROAD,BROWN,...,SMOOTH,ORANGE,ORANGE,PARTIAL,BROWN,ONE,PENDANT,ORANGE,CLUSTERED,LEAVES
8414,EDIBLE,KNOBBED,SMOOTH,BROWN,NO,NONE,ATTACHED,CLOSE,BROAD,BROWN,...,SMOOTH,ORANGE,ORANGE,PARTIAL,BROWN,ONE,PENDANT,BUFF,SEVERAL,LEAVES


# Naive Bayes Passo a Passo
Vamos agora ver como faríamos para implementar o algoritimo do Naive Bayes usando 3 atributos, são eles: formato do chapéu (cap-shape), cor do chapéu (cap-color), habitat.

Para isso, precisaremos calcular a probabilidade condicionada do atributo assumir certo valor quando o cogumelo é tóxico e a probabilidade condicionada desse mesmo atributo assumir certo valor dado que o cogumelo é comestível. Esse cálculo deve ser feito para todos os valores que o atributo em questão pode assumir, tomando como exemplo o formato do chapéu, temos os seguintes valores: bell, conical, convex, flat, knobbed e sunken.

Além disso, esse processo deve ser repitido para todos os atributos considerados, no nosso caso estaremos considerando apenas 3, ditos acima. 

### Probablidade do cogumelo ser tóxico

In [ ]:
fieldQtd = [0, 0]
values = list(datasetExt['classe'].values)  #Guarda em uma lista os valores da classe seleconada

fieldNames = ['edible', 'poisonous']

for x in values:
  if x == 'EDIBLE': 
    fieldQtd[0]+=1;
  elif x == 'POISONOUS':
    fieldQtd[1]+=1;

#Exibindo valores
totalQtd = len(values)
print('total:', totalQtd)
edibleQtd = fieldQtd[0]
print('edible:', edibleQtd)
poisonousQtd = fieldQtd[1]
print('poisonous:',poisonousQtd)
P_poisonous = fieldQtd[1] / len(values)
print('P_poisonous:', round(P_poisonous, 2)*100,'%')
P_edible = edibleQtd / totalQtd;
print('P_edible:', round(P_edible, 2)*100,'%')

total: 8416
edible: 4488
poisonous: 3928
P_poisonous: 47.0 %
P_edible: 53.0 %


### Probabilidade condicionada do formato de chapéu quando o cogumelo for toxico

In [ ]:
from attr import field


fieldQtd = [0, 0, 0, 0, 0, 0]
valuesShape = list(datasetExt['cap-shape'].values) 
valuesClasse = list(datasetExt['classe'].values)

for x, y in zip(valuesShape, valuesClasse):
  if x == 'BELL' and y == 'POISONOUS' : 
    fieldQtd[0]+=1;
  elif x == 'CONICAL' and y == 'POISONOUS':
    fieldQtd[1]+=1;
  elif x == 'CONVEX' and y == 'POISONOUS':
    fieldQtd[2]+=1;
  elif x == 'FLAT' and y == 'POISONOUS':
    fieldQtd[3]+=1;
  elif x == 'KNOBBED' and y == 'POISONOUS':
    fieldQtd[4]+=1;
  elif x == 'SUNKEN' and y == 'POISONOUS':
    fieldQtd[5]+=1;

P_bellPoisonous = fieldQtd[0] / poisonousQtd
P_conicalPoisonous = fieldQtd[1] / poisonousQtd
P_convexPoisonous = fieldQtd[2] / poisonousQtd
P_flatPoisonous = fieldQtd[3] / poisonousQtd
P_knobbedPoisonous = fieldQtd[4] / poisonousQtd #Probabilidade condicionada de um cogumelo ter tampa com botão quando for toxico
P_sunkenPoisonous = fieldQtd[5] / poisonousQtd

### Probabilidade condicionada do formato de chapéu quando for comestível

In [ ]:
from attr import field


fieldQtd = [0, 0, 0, 0, 0, 0]
valuesShape = list(datasetExt['cap-shape'].values) 
valuesClasse = list(datasetExt['classe'].values)

for x, y in zip(valuesShape, valuesClasse):
  if x == 'BELL' and y == 'EDIBLE' : 
    fieldQtd[0]+=1;
  elif x == 'CONICAL' and y == 'EDIBLE':
    fieldQtd[1]+=1;
  elif x == 'CONVEX' and y == 'EDIBLE':
    fieldQtd[2]+=1;
  elif x == 'FLAT' and y == 'EDIBLE':
    fieldQtd[3]+=1;
  elif x == 'KNOBBED' and y == 'EDIBLE':
    fieldQtd[4]+=1;
  elif x == 'SUNKEN' and y == 'EDIBLE':
    fieldQtd[5]+=1;

P_bellEdible = fieldQtd[0] / edibleQtd
P_conicalEdible = fieldQtd[1] / edibleQtd
P_convexEdible = fieldQtd[2] / edibleQtd
P_flatEdible = fieldQtd[3] / edibleQtd
P_knobbedEdible = fieldQtd[4] / edibleQtd
P_sunkenEdible = fieldQtd[5] / edibleQtd

### Probabilidade condicionada da cor de chapéu quando for tóxico

In [ ]:
fieldQtd = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
valuesColor = list(datasetExt['cap-color'].values) 
valuesClasse = list(datasetExt['classe'].values)

for x, y in zip(valuesColor, valuesClasse):
    if x == 'BROWN' and y == 'POISONOUS' : 
        fieldQtd[0]+=1;
    elif x == 'BUFF' and y == 'POISONOUS':
        fieldQtd[1]+=1;
    elif x == 'CINNAMON' and y == 'POISONOUS':
        fieldQtd[2]+=1;
    elif x == 'GRAY' and y == 'POISONOUS':
        fieldQtd[3]+=1;
    elif x == 'GREEN' and y == 'POISONOUS':
        fieldQtd[4]+=1;
    elif x == 'PINK' and y == 'POISONOUS':
        fieldQtd[5]+=1;
    elif x == 'PURPLE' and y == 'POISONOUS':
        fieldQtd[6]+=1;
    elif x == 'RED' and y == 'POISONOUS':
        fieldQtd[7]+=1;
    elif x == 'WHITE' and y == 'POISONOUS':
        fieldQtd[8]+=1;
    elif x == 'YELLOW' and y == 'POISONOUS':
        fieldQtd[9]+=1;

P_brownPoisonous = fieldQtd[0] / poisonousQtd
P_buffPoisonous = fieldQtd[1] / poisonousQtd
P_cinnamonPoisonous = fieldQtd[2] / poisonousQtd
P_grayPoisonous = fieldQtd[3] / poisonousQtd
P_greenPoisonous = fieldQtd[4] / poisonousQtd
P_pinkPoisonous = fieldQtd[5] / poisonousQtd
P_purplePoisonous = fieldQtd[6] / poisonousQtd
P_redPoisonous = fieldQtd[7] / poisonousQtd
P_whitePoisonous = fieldQtd[8] / poisonousQtd
P_yellowPoisonous = fieldQtd[9] / poisonousQtd

### Probabilidade condicionada da cor de chapéu quando for comestível

In [ ]:
fieldQtd = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
valuesColor = list(datasetExt['cap-color'].values) 
valuesClasse = list(datasetExt['classe'].values)

for x, y in zip(valuesColor, valuesClasse):
    if x == 'BROWN' and y == 'EDIBLE' : 
        fieldQtd[0]+=1;
    elif x == 'BUFF' and y == 'EDIBLE':
        fieldQtd[1]+=1;
    elif x == 'CINNAMON' and y == 'EDIBLE':
        fieldQtd[2]+=1;
    elif x == 'GRAY' and y == 'EDIBLE':
        fieldQtd[3]+=1;
    elif x == 'GREEN' and y == 'EDIBLE':
        fieldQtd[4]+=1;
    elif x == 'PINK' and y == 'EDIBLE':
        fieldQtd[5]+=1;
    elif x == 'PURPLE' and y == 'EDIBLE':
        fieldQtd[6]+=1;
    elif x == 'RED' and y == 'EDIBLE':
        fieldQtd[7]+=1;
    elif x == 'WHITE' and y == 'EDIBLE':
        fieldQtd[8]+=1;
    elif x == 'YELLOW' and y == 'EDIBLE':
        fieldQtd[9]+=1;

P_brownEdible = fieldQtd[0] / edibleQtd
P_buffEdible = fieldQtd[1] / edibleQtd
P_cinnamonEdible = fieldQtd[2] / edibleQtd
P_grayEdible = fieldQtd[3] / edibleQtd
P_greenEdible = fieldQtd[4] / edibleQtd
P_pinkEdible = fieldQtd[5] / edibleQtd
P_purpleEdible = fieldQtd[6] / edibleQtd
P_redEdible = fieldQtd[7] / edibleQtd
P_whiteEdible = fieldQtd[8] / edibleQtd
P_yellowEdible = fieldQtd[9] / edibleQtd

### Probabilidade condicionada do habitat quando for tóxico

In [ ]:
fieldQtd = [0, 0, 0, 0, 0, 0, 0]
valuesColor = list(datasetExt['habitat'].values) 
valuesClasse = list(datasetExt['classe'].values)

for x, y in zip(valuesColor, valuesClasse):
    if x == 'GRASSES' and y == 'POISONOUS' : 
        fieldQtd[0]+=1;
    elif x == 'LEAVES' and y == 'POISONOUS':
        fieldQtd[1]+=1;
    elif x == 'MEADOWS' and y == 'POISONOUS':
        fieldQtd[2]+=1;
    elif x == 'PATHS' and y == 'POISONOUS':
        fieldQtd[3]+=1;
    elif x == 'URBAN' and y == 'POISONOUS':
        fieldQtd[4]+=1;
    elif x == 'WASTE' and y == 'POISONOUS':
        fieldQtd[5]+=1;
    elif x == 'WOODS' and y == 'POISONOUS':
        fieldQtd[6]+=1;

P_grassesPoisonous = fieldQtd[0] / poisonousQtd 
P_leavesPoisonous = fieldQtd[1] / poisonousQtd 
P_meadowsPoisonous = fieldQtd[2] / poisonousQtd 
P_pathsPoisonous = fieldQtd[3] / poisonousQtd 
P_urbanPoisonous = fieldQtd[4] / poisonousQtd 
P_wastePoisonous = fieldQtd[5] / poisonousQtd 
P_woodsPoisonous = fieldQtd[6] / poisonousQtd 

### Probabilidade condicionada do habitat quando for comestível

In [ ]:
fieldQtd = [0, 0, 0, 0, 0, 0, 0]
valuesColor = list(datasetExt['habitat'].values) 
valuesClasse = list(datasetExt['classe'].values)

for x, y in zip(valuesColor, valuesClasse):
    if x == 'GRASSES' and y == 'EDIBLE' : 
        fieldQtd[0]+=1;
    elif x == 'LEAVES' and y == 'EDIBLE':
        fieldQtd[1]+=1;
    elif x == 'MEADOWS' and y == 'EDIBLE':
        fieldQtd[2]+=1;
    elif x == 'PATHS' and y == 'EDIBLE':
        fieldQtd[3]+=1;
    elif x == 'URBAN' and y == 'EDIBLE':
        fieldQtd[4]+=1;
    elif x == 'WASTE' and y == 'EDIBLE':
        fieldQtd[5]+=1;
    elif x == 'WOODS' and y == 'EDIBLE':
        fieldQtd[6]+=1;

P_grassesEdible = fieldQtd[0] / edibleQtd 
P_leavesEdible = fieldQtd[1] / edibleQtd 
P_meadowsEdible = fieldQtd[2] / edibleQtd 
P_pathsEdible = fieldQtd[3] / edibleQtd 
P_urbanEdible = fieldQtd[4] / edibleQtd 
P_wasteEdible = fieldQtd[5] / edibleQtd 
P_woodsEdible = fieldQtd[6] / edibleQtd 

## Naive Bayes

Caso em que o cogumelo tem os seguintes valores:  
formato de chapéu: knobbed  
cor do chapéu: red  
habitat: paths.

In [ ]:
P_poisonousCondicoes = P_knobbedPoisonous*P_pathsPoisonous*P_redPoisonous*P_poisonous 
P_edibleCondicoes = P_knobbedEdible*P_pathsEdible*P_redEdible*P_edible 
print('P_poisonousCondicoes:', P_poisonousCondicoes)
print('P_edibleCondicoes', P_edibleCondicoes)
if(P_poisonousCondicoes > P_edibleCondicoes):
    print('Cogumelo tóxico')
else:
    print('Cogumelo comestível')


P_poisonousCondicoes: 0.00410726658409919
P_edibleCondicoes 0.00011814737521819525
Cogumelo tóxico


### Agora para o caso geral:

In [ ]:
capShape = input("Digite o formato do chapeu: ")
habitat = input("Digite o habitat: ")
capColor = input("Digite a cor do chapeu: ")

if(capColor == 'BROWN'):
    P_capColorPoisonous = P_brownPoisonous
    P_capColorEdible = P_brownEdible
    if(habitat == 'GRASSES'):
        P_habitatPoisonous = P_grassesPoisonous
        P_habitatEdible = P_grassesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'LEAVES'):
        P_habitatPoisonous = P_leavesPoisonous
        P_habitatEdible = P_leavesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'MEADOWS'):
        P_habitatPoisonous = P_meadowsPoisonous
        P_habitatEdible = P_meadowsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'PATHS'):
        P_habitatPoisonous = P_pathsPoisonous
        P_habitatEdible = P_pathsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'URBAN'):
        P_habitatPoisonous = P_urbanPoisonous
        P_habitatEdible = P_urbanEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WASTE'):
        P_habitatPoisonous = P_wastePoisonous
        P_habitatEdible = P_wasteEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WOODS'):
        P_habitatPoisonous = P_woodsPoisonous
        P_habitatEdible = P_woodsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
elif(capColor == 'BUFF'):
    P_capColorPoisonous = P_buffPoisonous
    P_capColorEdible = P_buffEdible
    if(habitat == 'GRASSES'):
        P_habitatPoisonous = P_grassesPoisonous
        P_habitatEdible = P_grassesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'LEAVES'):
        P_habitatPoisonous = P_leavesPoisonous
        P_habitatEdible = P_leavesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'MEADOWS'):
        P_habitatPoisonous = P_meadowsPoisonous
        P_habitatEdible = P_meadowsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'PATHS'):
        P_habitatPoisonous = P_pathsPoisonous
        P_habitatEdible = P_pathsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'URBAN'):
        P_habitatPoisonous = P_urbanPoisonous
        P_habitatEdible = P_urbanEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WASTE'):
        P_habitatPoisonous = P_wastePoisonous
        P_habitatEdible = P_wasteEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WOODS'):
        P_habitatPoisonous = P_woodsPoisonous
        P_habitatEdible = P_woodsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
elif(capColor == 'CINNAMON'):
    P_capColorPoisonous = P_cinnamonPoisonous
    P_capColorEdible = P_cinnamonEdible
    if(habitat == 'GRASSES'):
        P_habitatPoisonous = P_grassesPoisonous
        P_habitatEdible = P_grassesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'LEAVES'):
        P_habitatPoisonous = P_leavesPoisonous
        P_habitatEdible = P_leavesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'MEADOWS'):
        P_habitatPoisonous = P_meadowsPoisonous
        P_habitatEdible = P_meadowsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'PATHS'):
        P_habitatPoisonous = P_pathsPoisonous
        P_habitatEdible = P_pathsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'URBAN'):
        P_habitatPoisonous = P_urbanPoisonous
        P_habitatEdible = P_urbanEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WASTE'):
        P_habitatPoisonous = P_wastePoisonous
        P_habitatEdible = P_wasteEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WOODS'):
        P_habitatPoisonous = P_woodsPoisonous
        P_habitatEdible = P_woodsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
elif(capColor == 'GRAY'):
    P_capColorPoisonous = P_grayPoisonous
    P_capColorEdible = P_grayEdible
    if(habitat == 'GRASSES'):
        P_habitatPoisonous = P_grassesPoisonous
        P_habitatEdible = P_grassesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'LEAVES'):
        P_habitatPoisonous = P_leavesPoisonous
        P_habitatEdible = P_leavesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'MEADOWS'):
        P_habitatPoisonous = P_meadowsPoisonous
        P_habitatEdible = P_meadowsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'PATHS'):
        P_habitatPoisonous = P_pathsPoisonous
        P_habitatEdible = P_pathsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'URBAN'):
        P_habitatPoisonous = P_urbanPoisonous
        P_habitatEdible = P_urbanEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WASTE'):
        P_habitatPoisonous = P_wastePoisonous
        P_habitatEdible = P_wasteEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WOODS'):
        P_habitatPoisonous = P_woodsPoisonous
        P_habitatEdible = P_woodsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
elif(capColor == 'GREEN'):
    P_capColorPoisonous = P_greenPoisonous
    P_capColorEdible = P_greenEdible
    if(habitat == 'GRASSES'):
        P_habitatPoisonous = P_grassesPoisonous
        P_habitatEdible = P_grassesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'LEAVES'):
        P_habitatPoisonous = P_leavesPoisonous
        P_habitatEdible = P_leavesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'MEADOWS'):
        P_habitatPoisonous = P_meadowsPoisonous
        P_habitatEdible = P_meadowsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'PATHS'):
        P_habitatPoisonous = P_pathsPoisonous
        P_habitatEdible = P_pathsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'URBAN'):
        P_habitatPoisonous = P_urbanPoisonous
        P_habitatEdible = P_urbanEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WASTE'):
        P_habitatPoisonous = P_wastePoisonous
        P_habitatEdible = P_wasteEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WOODS'):
        P_habitatPoisonous = P_woodsPoisonous
        P_habitatEdible = P_woodsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
elif(capColor == 'PINK'):
    P_capColorPoisonous = P_pinkPoisonous
    P_capColorEdible = P_pinkEdible
    if(habitat == 'GRASSES'):
        P_habitatPoisonous = P_grassesPoisonous
        P_habitatEdible = P_grassesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'LEAVES'):
        P_habitatPoisonous = P_leavesPoisonous
        P_habitatEdible = P_leavesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'MEADOWS'):
        P_habitatPoisonous = P_meadowsPoisonous
        P_habitatEdible = P_meadowsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'PATHS'):
        P_habitatPoisonous = P_pathsPoisonous
        P_habitatEdible = P_pathsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'URBAN'):
        P_habitatPoisonous = P_urbanPoisonous
        P_habitatEdible = P_urbanEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WASTE'):
        P_habitatPoisonous = P_wastePoisonous
        P_habitatEdible = P_wasteEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WOODS'):
        P_habitatPoisonous = P_woodsPoisonous
        P_habitatEdible = P_woodsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
elif(capColor == 'PURPLE'):
    P_capColorPoisonous = P_purplePoisonous
    P_capColorEdible = P_purpleEdible
    if(habitat == 'GRASSES'):
        P_habitatPoisonous = P_grassesPoisonous
        P_habitatEdible = P_grassesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'LEAVES'):
        P_habitatPoisonous = P_leavesPoisonous
        P_habitatEdible = P_leavesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'MEADOWS'):
        P_habitatPoisonous = P_meadowsPoisonous
        P_habitatEdible = P_meadowsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'PATHS'):
        P_habitatPoisonous = P_pathsPoisonous
        P_habitatEdible = P_pathsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'URBAN'):
        P_habitatPoisonous = P_urbanPoisonous
        P_habitatEdible = P_urbanEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WASTE'):
        P_habitatPoisonous = P_wastePoisonous
        P_habitatEdible = P_wasteEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WOODS'):
        P_habitatPoisonous = P_woodsPoisonous
        P_habitatEdible = P_woodsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
elif(capColor == 'RED'):
    P_capColorPoisonous = P_redPoisonous
    P_capColorEdible = P_redEdible
    if(habitat == 'GRASSES'):
        P_habitatPoisonous = P_grassesPoisonous
        P_habitatEdible = P_grassesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'LEAVES'):
        P_habitatPoisonous = P_leavesPoisonous
        P_habitatEdible = P_leavesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'MEADOWS'):
        P_habitatPoisonous = P_meadowsPoisonous
        P_habitatEdible = P_meadowsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'PATHS'):
        P_habitatPoisonous = P_pathsPoisonous
        P_habitatEdible = P_pathsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'URBAN'):
        P_habitatPoisonous = P_urbanPoisonous
        P_habitatEdible = P_urbanEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WASTE'):
        P_habitatPoisonous = P_wastePoisonous
        P_habitatEdible = P_wasteEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WOODS'):
        P_habitatPoisonous = P_woodsPoisonous
        P_habitatEdible = P_woodsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
elif(capColor == 'WHITE'):
    P_capColorPoisonous = P_whitePoisonous
    P_capColorEdible = P_whiteEdible
    if(habitat == 'GRASSES'):
        P_habitatPoisonous = P_grassesPoisonous
        P_habitatEdible = P_grassesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'LEAVES'):
        P_habitatPoisonous = P_leavesPoisonous
        P_habitatEdible = P_leavesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'MEADOWS'):
        P_habitatPoisonous = P_meadowsPoisonous
        P_habitatEdible = P_meadowsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'PATHS'):
        P_habitatPoisonous = P_pathsPoisonous
        P_habitatEdible = P_pathsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'URBAN'):
        P_habitatPoisonous = P_urbanPoisonous
        P_habitatEdible = P_urbanEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WASTE'):
        P_habitatPoisonous = P_wastePoisonous
        P_habitatEdible = P_wasteEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WOODS'):
        P_habitatPoisonous = P_woodsPoisonous
        P_habitatEdible = P_woodsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
elif(capColor == 'YELLOW'):
    P_capColorPoisonous = P_yellowPoisonous
    P_capColorEdible = P_yellowEdible
    if(habitat == 'GRASSES'):
        P_habitatPoisonous = P_grassesPoisonous
        P_habitatEdible = P_grassesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'LEAVES'):
        P_habitatPoisonous = P_leavesPoisonous
        P_habitatEdible = P_leavesEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'MEADOWS'):
        P_habitatPoisonous = P_meadowsPoisonous
        P_habitatEdible = P_meadowsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'PATHS'):
        P_habitatPoisonous = P_pathsPoisonous
        P_habitatEdible = P_pathsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'URBAN'):
        P_habitatPoisonous = P_urbanPoisonous
        P_habitatEdible = P_urbanEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WASTE'):
        P_habitatPoisonous = P_wastePoisonous
        P_habitatEdible = P_wasteEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
    elif(habitat == 'WOODS'):
        P_habitatPoisonous = P_woodsPoisonous
        P_habitatEdible = P_woodsEdible
        if(capShape == 'BELL'):
            P_poisonousCondicoes = P_bellPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_bellEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONICAL'):
            P_poisonousCondicoes = P_conicalPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_conicalEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'CONVEX'):
            P_poisonousCondicoes = P_convexPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_convexEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'FLAT'):
            P_poisonousCondicoes = P_flatPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_flatEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'KNOBBED'):
            P_poisonousCondicoes = P_knobbedPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_knobbedEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')
        elif(capShape == 'SUNKEN'):
            P_poisonousCondicoes = P_sunkenPoisonous*P_habitatPoisonous*P_capColorPoisonous*P_poisonous 
            P_edibleCondicoes = P_sunkenEdible*P_habitatEdible*P_capColorEdible*P_edible 
            print('P_poisonousCondicoes:', P_poisonousCondicoes)
            print('P_edibleCondicoes', P_edibleCondicoes)
            if(P_poisonousCondicoes > P_edibleCondicoes):
                print('Cogumelo tóxico')
            else:
                print('Cogumelo comestível')



Digite o formato do chapeu: CONVEX
Digite o habitat: WOODS
Digite a cor do chapeu: YELLOW
P_poisonousCondicoes: 0.011340555230139505
P_edibleCondicoes 0.00924494074497005
Cogumelo tóxico


### Verificando se os resultados são coerêntes:

In [ ]:
fieldQtd = [0, 0]
valuesColor = list(datasetExt['cap-color'].values) 
valuesHabitat = list(datasetExt['habitat'].values) 
valuesShape = list(datasetExt['cap-shape'].values)
valuesClasse = list(datasetExt['classe'].values)

for x, y, z, w in zip(valuesColor, valuesHabitat, valuesShape, valuesClasse):
    if x == 'YELLOW' and y == 'WOODS' and z == 'CONVEX' and w == 'EDIBLE' : 
        fieldQtd[0]+=1
    elif x == 'YELLOW' and y == 'WOODS' and z == 'CONVEX'and w == 'POISONOUS':
        fieldQtd[1]+=1

print('toxicos', fieldQtd[1])
print('comestiveis', fieldQtd[0])

toxicos 112
comestiveis 24


In [ ]:
fieldQtd = [0, 0]
valuesColor = list(datasetExt['cap-color'].values) 
valuesHabitat = list(datasetExt['habitat'].values) 
valuesShape = list(datasetExt['cap-shape'].values)
valuesClasse = list(datasetExt['classe'].values)

for x, y, z, w in zip(valuesColor, valuesHabitat, valuesShape, valuesClasse):
    if x == 'WHITE' and y == 'URBAN' and z == 'CONVEX' and w == 'EDIBLE' : 
        fieldQtd[0]+=1
    elif x == 'WHITE' and y == 'URBAN' and z == 'CONVEX'and w == 'POISONOUS':
        fieldQtd[1]+=1

print('toxicos', fieldQtd[1])
print('comestiveis', fieldQtd[0])

toxicos 56
comestiveis 0
